In [ ]:
!pip install -q datasets==2.13.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00


Following code comes from https://www.kaggle.com/code/mnavaidd/caption-generation-using-blip-from-chest-x-ray

In [ ]:
#Data Processing/Management: pandas, datasets, itertools, tqdm, json
#ML/DL: transformers, torch
#Image Processing/Transformations: Resize (torchvision.transforms), albumentations, cv2 (OpenCV), PIL.image
#Utility/System Management: os, gc, numpy, shutil, requests
#Visualization: matplotlib.pyplot

import pandas as pd
from datasets import load_dataset
# import transformers
#from transformers import BlipProcessor, BlipForImageTextRetrieval,BlipForConditionalGeneration, AutoProcessor
from transformers import (
    AdamW,
    AutoModelForCausalLM,
    AutoProcessor,
    get_scheduler
)
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import os

import gc
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import albumentations as A
import cv2
import shutil
import json
from PIL import Image
import requests
from matplotlib import pyplot as plt

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
#From ROCO Dataset, only working with radiologytraindata.csv
df_train = pd.read_csv('radiologytraindata.csv', delimiter=',')
df_train.dataframeName = 'radiologytestdata.csv'
rows, cols = df_train.shape
print(f'There are {rows} rows and {cols} columns')

There are 65450 rows and 3 columns


In [ ]:
df_train.head()

,id,name,caption
0,ROCO_00002,PMC4083729_AMHSR-4-14-g002.jpg,Computed tomography scan in axial view showin...
1,ROCO_00003,PMC2837471_IJD2009-150251.001.jpg,Bacterial contamination occurred after comple...
2,ROCO_00004,PMC2505281_11999_2007_30_Fig6_HTML.jpg,The patient had residual paralysis of the han...
3,ROCO_00005,PMC3745845_IJD2013-683423.005.jpg,Panoramic radiograph after immediate loading.\n
4,ROCO_00007,PMC4917066_amjcaserep-17-301-g001.jpg,Plain abdomen x-ray: Multiple air levels at t...


In [ ]:
mask = df_train['caption'].str.contains('chest x-ray', case=False)
filtered_df = df_train[mask]
rows, cols = filtered_df.shape
print(f'There are {rows} rows and {cols} columns')
filtered_df.head()

There are 1735 rows and 3 columns


,id,name,caption
69,ROCO_00087,PMC5144533_IJCCM-20-677-g002.jpg,"Chest X-ray, which confirmed the position of ..."
141,ROCO_00172,PMC4863054_ir-14-187-g002.jpg,Chest X-ray findings. Chest radiograph reveal...
180,ROCO_00232,PMC4093973_IJCIIS-4-186-g001.jpg,"Chest X-ray, PA, showing the position of the ..."
215,ROCO_00274,PMC5616218_cureus-0009-00000001523-i01.jpg,Chest x-ray showing right-sided pneumothorax.\n
307,ROCO_00383,PMC5018069_gr1.jpg,Chest X-ray on the day of admission showing d...


In [ ]:
filtered_df['images'] = "radiology_train/" + filtered_df['name']
filtered_df.head()

<ipython-input-6-fd34c1155112>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['images'] = "radiology_train/" + filtered_df['name']


,id,name,caption,images
69,ROCO_00087,PMC5144533_IJCCM-20-677-g002.jpg,"Chest X-ray, which confirmed the position of ...",radiology_train/PMC5144533_IJCCM-20-677-g002.jpg
141,ROCO_00172,PMC4863054_ir-14-187-g002.jpg,Chest X-ray findings. Chest radiograph reveal...,radiology_train/PMC4863054_ir-14-187-g002.jpg
180,ROCO_00232,PMC4093973_IJCIIS-4-186-g001.jpg,"Chest X-ray, PA, showing the position of the ...",radiology_train/PMC4093973_IJCIIS-4-186-g001.jpg
215,ROCO_00274,PMC5616218_cureus-0009-00000001523-i01.jpg,Chest x-ray showing right-sided pneumothorax.\n,radiology_train/PMC5616218_cureus-0009-0000000...
307,ROCO_00383,PMC5018069_gr1.jpg,Chest X-ray on the day of admission showing d...,radiology_train/PMC5018069_gr1.jpg


Data is now filtered in filtered_df to only look at chest x-rays.

In [ ]:
# Formatting the data to fit Florence-2 requirements:
CHECKPOINT = "microsoft/Florence-2-base-ft"
REVISION = 'refs/pr/6'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True, revision=REVISION).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True, revision=REVISION)

In [ ]:
# text1 = filtered_df['caption'].iloc[0]
# # example: "/kaggle/input/roco-dataset/all_data/test/radiology/images/PMC1208923_1476-7120-3-19-3.jpg"
# #image_path = "all_data/train/radiology/images/" + filtered_df['name'].iloc[0]
# image_path = filtered_df['images'].iloc[0]
# #image_path = "radiology_train/" + filtered_df['name'].iloc[0]
# image1 = Image.open(image_path)

# inputs = processor(text=text1,
#                    images=image1,
#                    return_tensors="pt").to(DEVICE)

# inputs

In [ ]:
# The for-loop should go through all of the rows of filtered_df and add it to the inputs as a list
text_list = list()
image_path = list()
image_list = list()

for i in range(0, len(filtered_df)):
  text_list.append(filtered_df['caption'].iloc[i])
  image_path = filtered_df['images'].iloc[i]
  image_list.append(Image.open(image_path))



inputs = processor(text=text1,
                   images=image1,
                   return_tensors="pt").to(DEVICE)

text_list

[' Chest X-ray, which confirmed the position of guidewire, extending from the right internal jugular vein up to inferior vena cava\n']